# Sequential Channel Learning for Mobile Beam Tracking

In [4]:
# import packages
import os
import math
import time
import json
import random
import scipy.io as sio

from collections import OrderedDict

import numpy as np
import tensorflow as tf

# from seq2seq_model import Seq2SeqModel
from seq2seq_model_new import Seq2SeqModel

In [47]:
# Loading data
Data = sio.loadmat('./data/WI_MIMO_1d_28000MHz_HO_Beam1024_TT_sp2.mat')
X_train = Data['X_train']
y_train = Data['y_train']
X_test = Data['X_test']
y_test = Data['y_test']

In [65]:
# Decoding parameters
test_config = {}

# Network parameters
test_config['cell_type'] = 'lstm'
test_config['attention_type'] =  'luong'
test_config['hidden_units'] = 256
test_config['depth'] =  2
test_config['embedding_size'] =  500
test_config['num_encoder_symbols'] = 10001
test_config['num_decoder_symbols'] = 1026
test_config['input_size'] = 128

test_config['use_residual'] =  False
test_config['attn_input_feeding'] =  False
test_config['use_dropout'] =  True
test_config['dropout_rate'] =  0.3
test_config['beam_width'] =  1
test_config['decode_batch_size'] =  80
test_config['max_decode_step'] = 1000
test_config['write_n_best'] =  False
test_config['model_path'] = 'model_i20o20_h128_2_e500_ttbs_atth_wihosp2/'
test_config['decode_input'] = 'data/seq_channel_learning'
test_config['decode_output'] = './result/SCL_infer/' + test_config['model_path']
test_config['model_file'] = test_config['model_path'] + 'scl.ckpt-12090'
# Training parameters
test_config['learning_rate'] = 0.0002
test_config['max_gradient_norm'] =  1.0
test_config['batch_size'] =  128
test_config['max_epochs'] = 100
test_config['max_batch'] = 10000
test_config['max_load_batches'] = 20
test_config['max_seq_length'] = 50
test_config['display_freq'] = 100
test_config['save_freq'] =  11500
test_config['valid_freq'] = 1150000
test_config['optimizer'] = 'adam'
test_config['model_name'] = 'translate.ckpt'
test_config['shuffle_each_epoch'] = True
test_config['sort_by_length'] = True
test_config['use_fp16'] = False

# Runtime parameters
test_config['allow_soft_placement'] = True
test_config['log_device_placement'] = False

# Channel Inference parameters
source_len = 20
target_len = 20
path_tic = 0
loc_tic = 0

In [2]:
def load_model(session, test_config):
    model = Seq2SeqModel(test_config, 'decode')
    ckpt = tf.train.get_checkpoint_state(test_config['model_path'])
    if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        print ('Reloading model parameters..')
        # model.restore(session, ckpt.model_checkpoint_path)
        model.restore(session, test_config['model_file'])
    else:
        raise ValueError(
            'No such file:[{}]'.format(test_config['model_path']))
    return model

# Preparation on Data
def prepare_train_batch(X,y,batch_size,ptic,ltic):
    N_path = np.shape(X)[1]
    flags = 0
    flags2 = 0
    X_batch = np.zeros([batch_size,source_len],dtype=np.int32)
    y_batch = np.zeros([batch_size,target_len],dtype=np.int32)
    if path_tic >= N_path:
            flags = 1
    else:
        Temp_X = X[0,ptic]
        Temp_X.shape = -1
        Temp_y = y[0,ptic]
        Temp_y.shape = -1
        for i in range(batch_size):
            N_sam = np.shape(Temp_X)[0]
            while (ltic + source_len + target_len) >= N_sam:
                ptic = ptic + 1
                if(ptic >= N_path):
                    if i==0:
                        flags = 1
                    else:
                        flags2 = i
                    break
                else:
                    ltic = 0
                    Temp_X = X[0,ptic]
                    Temp_y = y[0,ptic]
                    N_sam = np.shape(Temp_X)[0]
            if(flags==1 or flags2>0):
                break
            else:
                X_batch[i,:] = np.array(Temp_X[ltic:ltic+source_len]).reshape(-1)
                y_batch[i,:] = np.array(Temp_y[ltic+source_len:ltic+source_len+target_len]).reshape(-1)
                ltic = ltic + target_len
    if flags == 1:
        if flags2 == 0:   # No data samples
            return None,None,None,None,0,0
        else:            #last data samples
            X_batch = X_batch[0:flags2,:]
            y_batch = y_batch[0:flags2,:]
            X_length = [source_len for k in range(flags2)]       
            y_length = [target_len for k in range(flags2)]  
    else:
        X_length = [source_len for k in range(batch_size)]       
        y_length = [target_len for k in range(batch_size)]
    X_length = np.array(X_length)
    y_length = np.array(y_length)
    return X_batch,X_length,y_batch,y_length,ptic,ltic

# without using embedding layers for input
def prepare_train_batch_new(X,y,batch_size,ptic,ltic):
    N_path = np.shape(X)[1]
    N_BS = np.shape(X[0,0])[1]
    flags = 0
    flags2 = 0
    X_batch = np.zeros([batch_size,source_len,N_BS],dtype=np.float32)
    y_batch = np.zeros([batch_size,target_len],dtype=np.int32)
    if path_tic >= N_path:
            flags = 1
    else:
        Temp_X = X[0,ptic]
        # Temp_X.shape = -1
        Temp_y = y[0,ptic]
        # Temp_y.shape = -1
        for i in range(batch_size):
            N_sam = np.shape(Temp_X)[0]
            while (ltic + source_len + target_len) >= N_sam:
                ptic = ptic + 1
                if(ptic >= N_path):
                    if i==0:
                        flags = 1
                    else:
                        flags2 = i
                    break
                else:
                    ltic = 0
                    Temp_X = X[0,ptic]
                    Temp_y = y[0,ptic]
                    N_sam = np.shape(Temp_X)[0]
            if(flags==1 or flags2>0):
                break
            else:
                X_batch[i,:,:] = np.array(Temp_X[ltic:ltic+source_len,:])
                y_batch[i,:] = np.array(Temp_y[ltic+source_len:ltic+source_len+target_len]).reshape(-1)
                ltic = ltic + source_len
    if flags == 1:
        if flags2 == 0:   # No data samples
            return None,None,None,None,0,0
        else:            #last data samples
            X_batch = X_batch[0:flags2,:]
            y_batch = y_batch[0:flags2,:]
            X_length = [source_len for k in range(flags2)]       
            y_length = [target_len for k in range(flags2)]  
    else:
        X_length = [source_len for k in range(batch_size)]       
        y_length = [target_len for k in range(batch_size)]
    X_length = np.array(X_length)
    y_length = np.array(y_length)
    y_batch = y_batch+1
    return X_batch,X_length,y_batch,y_length,ptic,ltic

def prepare_train_batch_HO(X,y,batch_size,ptic,ltic):
    N_path = np.shape(X)[1]
    N_BS = np.shape(X[0,0])[1]
    flags = 0
    flags2 = 0
    X_batch = np.zeros([batch_size,source_len,N_BS],dtype=np.float32)
    y_batch = np.zeros([batch_size,target_len],dtype=np.int32)
    y2_batch = np.zeros([batch_size,target_len,N_pattern],dtype=np.float32)
    if path_tic >= N_path:
            flags = 1
    else:
        Temp_X = X[0,ptic]
        # Temp_X.shape = -1
        Temp_y = y[0,ptic]
        Temp_y2 = y2[0,ptic]
        # Temp_y.shape = -1
        for i in range(batch_size):
            N_sam = np.shape(Temp_X)[0]
            while (ltic + source_len + target_len) >= N_sam:
                ptic = ptic + 1
                if(ptic >= N_path):
                    if i==0:
                        flags = 1
                    else:
                        flags2 = i
                    break
                else:
                    ltic = 0
                    Temp_X = X[0,ptic]
                    Temp_y = y[0,ptic]
                    N_sam = np.shape(Temp_X)[0]
            if(flags==1 or flags2>0):
                break
            else:
                X_batch[i,:,:] = np.array(Temp_X[ltic:ltic+source_len,:])
                y_batch[i,:] = np.array(Temp_y[ltic+source_len:ltic+source_len+target_len]).reshape(-1)
                y2_batch[i,:,:] = np.array(Temp_y[ltic+source_len:ltic+source_len+target_len,:])
                ltic = ltic + source_len
    if flags == 1:
        if flags2 == 0:   # No data samples
            return None,None,None,None,0,0
        else:            #last data samples
            X_batch = X_batch[0:flags2,:]
            y_batch = y_batch[0:flags2,:]
            y2_batch = y_batch[0:flags2,:]
            X_length = [source_len for k in range(flags2)]       
            y_length = [target_len for k in range(flags2)]  
    else:
        X_length = [source_len for k in range(batch_size)]       
        y_length = [target_len for k in range(batch_size)]
    X_length = np.array(X_length)
    y_length = np.array(y_length)
    return X_batch,X_length,y_batch,y_length,ptic,ltic

In [66]:
path_tic = 0
loc_tic = 0
# creating output data
if not os.path.exists(test_config['decode_output']):
    os.makedirs(test_config['decode_output'])
tf.reset_default_graph()
# Initiate TF session
def angle_dis(a,b,N):
    a1 = -2*(a-1)
    a1 = a1 + 2*N*(a1<-N)
    b1 = -2*(b-1)
    b1 = b1 + 2*N*(b1<-N)
    c = abs(math.acos(a1/N)-math.acos(b1/N))
    return c

def angle_mse(a,b,N):
    a1 = -2*(a-2)
    a1 = a1 + 2*N*(a1<-N)
    b1 = -2*(b-2)
    b1 = b1 + 2*N*(b1<-N)
    c = abs(math.acos(a1/N)-math.acos(b1/N))
    c = c*c
    return c

with tf.Session(config=tf.ConfigProto(allow_soft_placement=test_config['allow_soft_placement'], 
    log_device_placement=test_config['log_device_placement'], gpu_options=tf.GPUOptions(allow_growth=True))) as sess:
    
    # Reload existing checkpoint
    model = load_model(sess, test_config)
    print('Decoding {}..'.format(test_config['decode_input']))
    dis = 0
    num = 0
    dis1 = np.zeros(target_len,dtype=np.float32)
    num1 = 0
    # for i_batch in range(test_config['max_batch']):
    for i_batch in range(1):
        path_tic_o = path_tic
        loc_tic_o = loc_tic
        source, source_len_g, target, target_len_g,path_tic,loc_tic = prepare_train_batch_new(X_test,y_test,test_config['decode_batch_size'],path_tic,loc_tic)
        # predicted_ids: GreedyDecoder; [batch_size, max_time_step, 1]
        # BeamSearchDecoder; [batch_size, max_time_step, beam_width]
        if source is None or target is None:
            print('No samples')
            break
        predicted_ids = model.predict(sess, encoder_inputs=source, 
                                      encoder_inputs_length=source_len_g)
        shape_target = np.shape(target)
        shape_predict = np.shape(predicted_ids)
        predicted = predicted_ids.reshape(shape_predict[0:2])
        predicted = predicted[:,0:np.shape(target)[1]]
        for i in range(np.shape(target)[0]):
            for j in range(0,np.shape(target)[1]):
                if(predicted[i,j]==1 and j > 0):
                    predicted[i,j] = predicted[i,j-1]
                dis  = dis + angle_mse(predicted[i,j],target[i,j],1024)
                dis1[j] = dis1[j] + angle_mse(predicted[i,j],target[i,j],1024)
        num = num + np.shape(target)[0]*np.shape(target)[1]
        num1 = num1 + np.shape(target)[0]
        np.savez(test_config['decode_output']+str(i_batch),path_tic = path_tic_o,loc_tic = loc_tic_o,results = predicted_ids)
    dis_mse = math.sqrt(dis/num)
    dis1_mse = np.zeros(target_len,dtype=np.float32)
    for i_deruation in range(target_len):
        dis1_mse[i_deruation] = math.sqrt(dis1[i_deruation]/num1)
    print('Decoding terminated')
    print('RMSE of angle:',dis_mse)
    print('RMSE of angle per ts:', dis1_mse)

building model..
building encoder..
building decoder and attention..
building greedy decoder..
Reloading model parameters..
INFO:tensorflow:Restoring parameters from model_i20o20_h128_2_e500_ttbs_atth_wihosp2/scl.ckpt-12090
model restored from model_i20o20_h128_2_e500_ttbs_atth_wihosp2/scl.ckpt-12090
Decoding data/seq_channel_learning..
Decoding terminated
RMSE of angle: 0.3745383916941015
RMSE of angle per ts: [ 0.27558655  0.24053803  0.28152052  0.2334199   0.317691    0.28708348
  0.32575306  0.34390476  0.41695324  0.42657349  0.40660655  0.38728252
  0.41247207  0.45880827  0.42826018  0.40453279  0.41936582  0.42903337
  0.43884411  0.42303661]


In [22]:
dis1_mse

array([ 0.13662356,  0.19877611,  0.17861727,  0.18322775,  0.14245971,
        0.19467416,  0.21998592,  0.23622383,  0.22993183,  0.15426639,
        0.20952325,  0.17539844,  0.21946575,  0.24989946,  0.18333901,
        0.13509357,  0.13639776,  0.24314374,  0.22335891,  0.17623675], dtype=float32)

In [43]:
predicted

array([[928, 928, 928, ..., 166, 166, 166],
       [925, 924, 926, ..., 929, 926, 926],
       [171, 170, 171, ..., 926,   1,   1],
       ..., 
       [990, 990, 993, ..., 990, 990, 990],
       [989, 989, 989, ..., 989, 989, 989],
       [987, 989, 989, ...,   1,   1,   1]])

In [42]:
target

array([[927, 927, 927, ..., 926, 926, 926],
       [925, 925, 925, ..., 170, 170, 170],
       [170, 171, 171, ..., 173, 173, 922],
       ..., 
       [989, 989, 989, ..., 989, 988, 988],
       [988, 988, 988, ..., 988, 988, 988],
       [119, 119, 119, ..., 987, 987, 987]])

Beam direction(rad)
Simulation Settings: model_i10o10_h128_2_nib

Train: 0.07973257308229005 

Average angle error per ts: [ 0.0794014  0.0795161  0.0795283  0.079619   0.0797241  0.0797927 0.079791   0.0798578  0.0799997 0.080082]


Test: 0.0794

Average angle error per ts:  [ 0.0792811  0.0793486  0.0793337  0.0793039  0.0794     0.0794374 0.0794082  0.0794173  0.0795315  0.0795499]

Simulation Settings: model_i10o20_h128_2_nib

model-0 0 epochs

Train:Average angle error: 0.6944357629593709
Average angle error per ts: [ 0.66718523  0.72369515  0.71231542  0.66350166  0.64444529  0.66052178
  0.6887362   0.7101336   0.7182222   0.713365    0.71818524  0.73866007
  0.74539693  0.72379678  0.69726404  0.67591962  0.66666731  0.66838475
  0.67322118  0.67916617]
  
Test:Average angle error: 0.6707483730751174
Average angle error per ts: [ 0.60797477  0.57340801  0.5732398   0.5687167   0.6503827   0.65234613
  0.66705489  0.69229811  0.71147984  0.6869607   0.66385263  0.67313379
  0.70035011  0.71898448  0.72381979  0.73454404  0.72117734  0.70499551
  0.69434232  0.69589555]
  

model-5380 10 epochs

Train:
Average angle error: 0.08570099013864824
Average angle error per ts: [ 0.0851762   0.08522608  0.08524533  0.08526518  0.0853069   0.08535923
  0.08545803  0.08548958  0.08549786  0.08557467  0.08563063  0.08574991
  0.08581949  0.08590564  0.08599267  0.08607334  0.08616547  0.08624317
  0.0863648   0.08642987]


Test Average angle error: 0.08520000839789366

Average angle error per ts: [ 0.08513188  0.08517958  0.08506685  0.08503445  0.08511009  0.08508803
  0.08517716  0.08512028  0.08511788  0.08505739  0.08504481  0.08520757
  0.08520835  0.08518627  0.08529633  0.08525585  0.08538452  0.08538754
  0.0854934   0.08545178]
  
model-10760 20 epochs

Train:
Average angle error: 0.0827663471420836
Average angle error per ts: [ 0.08223856  0.08234145  0.082371    0.08235672  0.0824486   0.08245246
  0.08249723  0.08258968  0.08266506  0.08269914  0.08273242  0.08285007
  0.08289707  0.08304337  0.08311055  0.08317149  0.08315391  0.08318375
  0.08320836  0.08327434]


Test
Average angle error: 0.08195105949962386
Average angle error per ts: [ 0.08180495  0.08188219  0.08178101  0.08176037  0.08189626  0.08181248
  0.08180017  0.0818921   0.08194748  0.08190646  0.08185168  0.08198519
  0.08200448  0.08198782  0.08211611  0.08212913  0.0821013   0.08208368
  0.08212261  0.08215526]


model-16380 30 epochs
train: 0.08107642133113113

Average angle error per ts:  [ 0.08048157  0.0805693   0.08062427  0.08065699  0.08070366  0.08076612
  0.08083243  0.08083595  0.08094989  0.08100486  0.08104184  0.08111501
  0.08116439  0.08130013  0.08143218  0.08152328  0.08156475  0.08155842
  0.08167306  0.08169322]
  
test:Average angle error: 0.08065237168005597

Average angle error per ts: [ 0.08057152  0.08061863  0.0805613   0.08048771  0.08058092  0.08059675
  0.08057364  0.08058482  0.08062769  0.08060081  0.08052798  0.08059341
  0.08064754  0.08061972  0.08083728  0.08083578  0.08085512  0.0807576
  0.08081182  0.08075854]
  

# Model model_i10h10_h128_2_e2000_nib/
# 16230
test: Average angle error: 0.07979312429618636
Average angle error per ts: [ 0.07991429  0.0794317   0.07913889  0.08017363  0.0804113   0.07987673
  0.08041405  0.07972659  0.0798196   0.07902446]
train: Average angle error: 0.035489935332863036
Average angle error per ts: [ 0.03502921  0.03436437  0.0351839   0.03550647  0.03559846  0.0354913
  0.03530699  0.03606808  0.03558343  0.03676715]
 
## scl.ckpt-21640
Average angle error: 0.03581494586005019
Average angle error per ts: [ 0.03526252  0.03467556  0.03532154  0.03540443  0.0360692   0.03587601
  0.03600241  0.03634839  0.03587168  0.03731775]
 
Average angle error: 0.07225993025332901
Average angle error per ts: [ 0.07255827  0.0718499   0.07191636  0.07240743  0.07279808  0.07240473
  0.07266887  0.07206371  0.07236081  0.07157113]
  
# Model model_i20h10_h128_2_e2000_nib/
## scl.ckpt-21520
train:  Average angle error: 0.06717606827146808
Average angle error per ts: [ 0.06691866  0.0670119   0.06708295  0.06711632  0.06712282  0.0672002
  0.06724576  0.06728819  0.06734806  0.06742359]
 
test: Average angle error: 0.07084651517962026
Average angle error per ts: [ 0.07077424  0.07081047  0.07085553  0.07072902  0.07090507  0.07084846
  0.07082728  0.070908    0.07080942  0.07099676]

# Model model_i20h20_h128_2_e2000_nib/
## scl.ckpt-13400
test: Average angle error: 0.07287015211099648
Average angle error per ts: [ 0.07242657  0.07242283  0.07250638  0.07248913  0.07257597  0.07261769
  0.07263568  0.0726455   0.07263714  0.07279912  0.0726986   0.07290357
  0.07292901  0.07288793  0.07301026  0.07306947  0.07315093  0.07315157
  0.07335317  0.07449199]

train:Average angle error: 0.06966608999768224
Average angle error per ts: [ 0.06870427  0.06879663  0.06892827  0.06903116  0.06905668  0.06915251
  0.0692246   0.06929958  0.06936064  0.06950492  0.0695955   0.06974683
  0.06978465  0.06991623  0.07002062  0.07012556  0.07025073  0.07033695
  0.07046653  0.07201347]


## scl.ckpt-10700
Average angle error: 0.07109828438797797
Average angle error per ts: [ 0.0702259   0.07033287  0.07049557  0.07054967  0.07063032  0.07072347
  0.07079262  0.07087973  0.07088592  0.07102083  0.07108232  0.07119366
  0.07123409  0.07137668  0.07147555  0.07153627  0.07160677  0.07167511
  0.07179083  0.07245494]

test:Average angle error: 0.07241568015935895
Average angle error per ts: [ 0.07206744  0.07206207  0.07216897  0.07212777  0.07231271  0.07228564
  0.07231334  0.07233255  0.07233784  0.07240763  0.07234161  0.07243709
  0.07247198  0.07243412  0.07249253  0.07257118  0.07265051  0.07263151
  0.07280552  0.07306114]
  


In [ ]:
i = 47
print(predicted[i,:])
print(target[i,:])

In [ ]:
source

In [ ]:
% matplotlib inline
from matplotlib import pyplot
train_res = np.array([0.0794014,0.0795161,0.0795283,0.079619,0.0797241,0.0797927,0.079791,0.0798578,0.0799997,0.080082])*180/(math.pi)
test_res = np.array([0.0792811,0.0793486,0.0793337,0.0793039,0.0794,0.0794374,0.0794082,0.0794173,0.0795315,0.0795499])*180/(math.pi)
x_ts = [i*5 for i in range(10)]
pyplot.xlabel('Delay(ms)')
pyplot.ylabel('Beam Direction Error(degree)')
plt1, = pyplot.plot(x_ts,train_res,'r-o')
plt2, = pyplot.plot(x_ts,test_res,'b-o')
pyplot.legend([plt1, plt2], ('Training set', 'test set'))